In [ ]:
mode = "png"

import matplotlib

font = {'family' : 'Dejavu Sans',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
from graspologic.simulations import sbm
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os

n = 100  # the number of nodes
M = 8  # the total number of networks
# human brains have homophilic block structure
Bhuman = np.array([[0.4, 0.02], [0.02, 0.4]])
# alien brains have a core-periphery block structure
Balien = np.array([[0.3, 0.1], [0.1, 0.05]])

# set seed for reproducibility
np.random.seed(123)
# generate 4 human and alien brain networks
A_humans = [sbm([n // 2, n // 2], Bhuman) for i in range(M // 2)]
A_aliens = [sbm([n // 2, n // 2], Balien) for i in range(M // 2)]
# concatenate list of human and alien networks
networks2 = A_humans + A_aliens

# 1 = left hemisphere, 2 = right hemisphere for node communities
le = LabelEncoder()
labels = np.repeat(["L", "R"], n//2)
zs = le.fit_transform(labels) + 1

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
from graphbook_code import heatmap

fig = plt.figure(figsize=(14,7))

grid1 = ImageGrid(fig, 121, (2, 2), axes_pad=.1, share_all=True)
grid2 = ImageGrid(fig, 122, (2, 2), axes_pad=.1, share_all=True)

for i, (axi, axj) in enumerate(zip(grid1, grid2)):
    hmn = heatmap(A_humans[i].astype(int), ax=axi, cbar=False)
    hma = heatmap(A_aliens[i].astype(int), ax=axj, cbar=False)

grid1.axes_all[0].set_title("(A) Human Brain Networks", fontsize=24, y=1, loc="left")
grid2.axes_all[0].set_title("(B) Alien Brain Networks", fontsize=24, y=1, loc="left")
fig.tight_layout()
fig.savefig("Figures/multi_ex.{}".format(mode))

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed as ase

# embed the first network with a seed for reproducibility
Xhat = ase(n_components=2, svd_seed=123).fit_transform(A_humans[0])

In [ ]:
# a rotation by 90 degrees
W = np.array([[0, -1], [1, 0]])
Yhat = Xhat @ W

# check that probability matrix is the same
np.allclose(Yhat @ Yhat.transpose(), Xhat @ Xhat.transpose())
# returns True

In [ ]:
from matplotlib import patches
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(Xhat[:,0], Xhat[:,1], color="gray", alpha=0.7)
axs[0].set_xlim([-1,1])
axs[0].set_ylim([-1,1])
axs[0].set_title("(A) $\\hat X$")
axs[0].set_xlabel("Dimension 1")
axs[0].set_ylabel("Dimension 2")

arrow_dat = {}
for z in np.unique(zs):
    center_unrot = Xhat[zs == z,:].mean(axis=0)
    center_rot = Yhat[zs == z,:].mean(axis=0)
    arrow_dat[z] = {"unrot": center_unrot, "rot": center_rot}

axs[1].scatter(Xhat[:,0], Xhat[:,1], color="gray", alpha=0.3)
axs[1].scatter(Yhat[:,0], Yhat[:,1], color="black", alpha=0.7)
axs[1].set_xlim([-1,1])
axs[1].set_ylim([-1,1])
axs[1].set_title("(B) $\\hat Y = \\hat X W$")
axs[1].set_xlabel("Dimension 1")
axs[1].set_ylabel("Dimension 2")

style = "->, head_length=10, head_width=5, scaleB=10, scaleA=10"
kw = dict(arrowstyle=style, color="gray", linewidth="2")

for z in arrow_dat.keys():
    arrow = arrow_dat[z]
    axs[1].add_patch(patches.FancyArrowPatch(arrow["unrot"]*.5, arrow["rot"]*.5, connectionstyle="arc3,rad=-.3", **kw))

fig.tight_layout()
fig.savefig("Figures/rotation.{}".format(mode))

In [ ]:
# embed the second network
Xhat2 = ase(n_components=2, svd_seed=123).fit_transform(A_humans[3])

In [ ]:
from graphbook_code import lpm_heatmap

fig, axs = plt.subplots(1, 4, figsize=(15, 5), gridspec_kw = {"width_ratios": [1, 1.27, .35, .45]})

heatmap(A_humans[0].astype(int), cbar=False, ax=axs[0],
        xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node",
        inner_hier_labels=zs, title="(A) First human network $A^{(1)}$")
heatmap(A_humans[1].astype(int), legend_title="Edge?", ax=axs[1],
        xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node",
        inner_hier_labels=zs, title="(B) Third human network $A^{(3)}$")

vmin = np.min([Xhat, Xhat2]); vmax = np.max([Xhat, Xhat2])
lpm_heatmap(Xhat, xtitle="Latent Dim.", xticks=[0.5, 1.5], xticklabels=[1, 2],
            yticks=[0.5, 49.5, 99.5], yticklabels=[1, 50, 100], ytitle="Node",
            ax=axs[2], cbar=False, vmin=vmin, vmax=vmax, title="(C) $\\hat X^{(1)}$")
lpm_heatmap(Xhat2, xtitle="Latent Dim.", xticks=[0.5, 1.5], xticklabels=[1, 2],
            yticks=[0.5, 49.5, 99.5], yticklabels=[1, 50, 100], ytitle="Node",
            ax=axs[3], vmin=vmin, vmax=vmax, title="(D) $\\hat X^{(3)}$", shrink=0.7)

fig.tight_layout()
fig.savefig("Figures/cmp_identical.{}".format(mode))

In [ ]:
# embed the first alien network
Xhat_alien = ase(n_components=2, svd_seed=123).fit_transform(A_aliens[0])

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 5), gridspec_kw = {"width_ratios": [1, 1.27, .35, .45]})

heatmap(A_humans[0].astype(int), cbar=False, ax=axs[0],
        xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node",
        inner_hier_labels=zs, title="(A) First human network $A^{(1)}$")
heatmap(A_aliens[0].astype(int), legend_title="Edge?", ax=axs[1],
        xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node",
        inner_hier_labels=zs, title="(B) First alien network $A^{(5)}$")

vmin = np.min([Xhat, Xhat2]); vmax = np.max([Xhat, Xhat2])
lpm_heatmap(Xhat, xtitle="Latent Dim.", xticks=[0.5, 1.5], xticklabels=[1, 2],
            yticks=[0.5, 49.5, 99.5], yticklabels=[1, 50, 100], ytitle="Node",
            ax=axs[2], cbar=False, vmin=vmin, vmax=vmax, title="(C) $\\hat X^{(1)}$")
lpm_heatmap(Xhat_alien, xtitle="Latent Dim.", xticks=[0.5, 1.5], xticklabels=[1, 2],
            yticks=[0.5, 49.5, 99.5], yticklabels=[1, 50, 100], ytitle="Node",
            ax=axs[3], vmin=vmin, vmax=vmax, title="(D) $\\hat X^{(5)}$", shrink=0.7)

fig.tight_layout()
fig.savefig("Figures/cmp_diss.{}".format(mode))

In [ ]:
# compute frob norm between first human and third human net
# estimated latent positions
dist_firsthum_thirdhum = np.linalg.norm(Xhat - Xhat2, ord="fro")
print("Frob. norm(first human, third human) = {:3f}".format(dist_firsthum_thirdhum))
#

# compute frob norm between first human and first alien net
# estimated latent positions
dist_firsthum_alien = np.linalg.norm(Xhat - Xhat_alien, ord="fro")
print("Frob. norm(first human, alien) = {:3f}".format(dist_firsthum_alien))

In [ ]:
from graspologic.embed import MultipleASE as mase

# Use mase to embed everything
mase = mase(n_components=2, svd_seed=123)
# fit_transform on the human and alien networks simultaneously
# + combines the two lists
latents_mase = mase.fit_transform(networks)

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed as ase

dhat = int(np.ceil(np.log2(n)))
# spectrally embed each network into ceil(log2(n)) dimensions with ASE
separate_embeddings = [ase(n_components=dhat, svd_seed=123).fit_transform(network) for network in networks]

In [ ]:
from graphbook_code import plot_latents

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

ax = plot_latents(separate_embeddings[0], labels=labels, title="(A) Human Embedding",
                  palette={"L": "#000000", "R": "#999999"}, s=30, ax=axs[0], legend=True)
ax.get_legend().remove()
plot_latents(separate_embeddings[3], labels=labels, title="(B) Alien Embedding",
                  palette={"L": "#000000", "R": "#999999"}, s=30, ax=axs[1], legend=True)
fig.tight_layout()
fig.savefig("Figures/mase_ase.{}".format(mode))

In [ ]:
# Concatenate your embeddings horizontally into a single n x Md matrix
joint_matrix = np.hstack(separate_embeddings)

In [ ]:
import matplotlib.gridspec as gridspec

# Create a figure
fig = plt.figure(figsize=(20, 15))

# Create a GridSpec object
gs = gridspec.GridSpec(2, 9)

vmin = np.min(joint_matrix); vmax=np.max(joint_matrix)

for i, network in enumerate(networks):
    ax = fig.add_subplot(gs[0, i])
    if i == 0:
        lpm_heatmap(separate_embeddings[i], ax=ax, vmin=vmin, vmax=vmax, cbar=False,
                    xticks=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5], xticklabels=[1,2,3,4,5,6,7],
                    yticks=[0.5, 49.5, 99.5], yticklabels=[1,50,100], ytitle="Node")
    else:
        lpm_heatmap(separate_embeddings[i], ax=ax, vmin=vmin, vmax=vmax, cbar=False,
                    xticks=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5], xticklabels=[1,2,3,4,5,6,7])

ax = fig.add_subplot(gs[1, :])
lpm_heatmap(joint_matrix, xticks=[0.5, 7.5, 14.5, 21.5, 28.5, 35.5, 42.5,49.5], xticklabels=[1,8,15,22,29,36,43,50],
            yticks=[0.5, 49.5, 99.5], yticklabels=[1,50,100], ytitle="Node", ax=ax)

fig.tight_layout()
fig.savefig("Figures/mase_embed_joint.svg")

In [ ]:
def unscaled_embed(X, d):
    U, s, Vt = np.linalg.svd(X)
    return U[:,0:d]

Vhat = unscaled_embed(joint_matrix, 2)

In [ ]:
# stack the networks into a numpy array
As_ar = np.asarray(networks)
# compute the scores
scores = Vhat.T @ As_ar @ Vhat

In [ ]:
# Create a figure
fig = plt.figure(figsize=(12, 10))

# Create a GridSpec object
gs = gridspec.GridSpec(2, 2)

ax = fig.add_subplot(gs[0, :])
plot_latents(Vhat, labels=labels, s=40, palette={"L": "#000000", "R": "#999999"},
            title="(A) Estimated shared latent positions")
ax.set_xlabel("Dimension 1")
ax.set_ylabel("Dimension 2")
ax.set_title("(A) Estimated shared latent positions", fontsize=22, pad=10, loc="left")

ax = fig.add_subplot(gs[1, 0])
smin=scores.min(); smax=scores.max()
heatmap(scores[0], vmin=smin, vmax=smax, annot=True, ax=ax, title="(B) Human 1 scores",
        xticks=[0.5, 1.5], xticklabels=[1, 2], yticks=[0.5, 1.5], yticklabels=[1, 2],
        xtitle="Dimension", ytitle="Dimension", cbar=False)
ax.set_title("(B) Human 1 scores", fontsize=22, pad=10)

ax = fig.add_subplot(gs[1, 1])
smin=scores.min(); smax=scores.max()
heatmap(scores[4], vmin=smin, vmax=smax, annot=True, ax=ax, title="(C) Alien 1 scores", legend_title="Score")
ax.set_title("(C) Alien 1 scores", fontsize=22, pad=10)
fig.tight_layout()
fig.savefig("Figures/mase_shared_lpm.{}".format(mode))

In [ ]:
from graphbook_code import generate_sbm_pmtx

Phum = generate_sbm_pmtx(zs, Bhuman)
Palien = generate_sbm_pmtx(zs, Balien)
Pests = Vhat @ scores @ Vhat.T

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

heatmap(Phum, ax=axs[0][0], title="(A) $P$ Human", inner_hier_labels=labels, vmin=0, vmax=1,
       xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node", legend_title="Probability")
heatmap(Pests[0], ax=axs[0][1], title="(B) $\\hat P$ Human 1", inner_hier_labels=labels, vmin=0, vmax=1,
       xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node", legend_title="Estimated Probability")

heatmap(Palien, ax=axs[1][0], title="(C) $P$ Alien", inner_hier_labels=labels, vmin=0, vmax=1,
       xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node", legend_title="Probability")
heatmap(Pests[4], ax=axs[1][1], title="(D) $\\hat P$ Alien 1", inner_hier_labels=labels, vmin=0, vmax=1,
       xticks=[0.5, 49.5, 99.5], xticklabels=[1, 50, 100], xtitle="Node", legend_title="Probability")

fig.tight_layout()
fig.savefig("Figures/mase_probs.{}".format(mode))

In [ ]:
import seaborn as sns
from graphbook_code import text
from graphbook_code import lined_heatmap, add_legend

# fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(2, 3)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
ax_omni = fig.add_subplot(gs[:, 1:])

# first two
omni_cmap = [[1,1,1], [0.5,0.5,0.5], [0,0,0]]
print(omni_cmap)
for i, (ax, data) in enumerate(zip([ax0, ax1], [networks[0], networks[1]])):
    title = r"(A) First network ($A^{(1)}$)" if i==0 else r"(B) Second network ($A^{(2)}$)"
    heatmap(data, ax=ax, cbar=False, title=title, inner_hier_labels=labels)

from graspologic.embed.omni import _get_omni_matrix
omni = _get_omni_matrix([networks[0], networks[1]])
# big one
hm = lined_heatmap(omni, ax=ax_omni, binary=False, cbar=False,
                   title="(C) Omnibus Matrix for first \nand second network",
                   center=None)

# outline
sns.despine(ax=ax_omni, top=False, bottom=False, left=False, right=False)

# separating lines
hm.vlines(len(omni)//2, 0, len(omni), colors="black", lw=.9, alpha=1)
hm.hlines(len(omni)//2, 0, len(omni), colors="black", lw=.9, alpha=1)
for i in [.25, .75]:
    hm.vlines(len(omni)*i, 0, len(omni), colors="black", lw=.9, linestyle="dashed", alpha=.6)
    hm.hlines(len(omni)*i, 0, len(omni), colors="black", lw=.9, linestyle="dashed", alpha=.6)
    
# text
t = text(r"$A^{(1)}$", .25, .75, ax=ax_omni)
t.set_bbox(dict(facecolor="white", edgecolor="white"))
t = text(r"$A^{(2)}$", .75, .25, ax=ax_omni)
t.set_bbox(dict(facecolor="white", edgecolor="white"))
t = text(r"$\frac{(A^{(2)} + A^{(1)}}{2}$", .25, .25, ax=ax_omni)
t.set_bbox(dict(facecolor="white", edgecolor="white"))
t = text(r"$\frac{(A^{(1)} + A^{(2)})}{2}$", .75, .75, ax=ax_omni)
t.set_bbox(dict(facecolor="white", edgecolor="white"))

fig.tight_layout()
fig.savefig("Figures/omni_ex.{}".format(mode))

In [ ]:
from graspologic.embed.omni import _get_omni_matrix
omni_mtx = _get_omni_matrix(networks)

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed as ase

dhat = int(np.ceil(np.log2(n)))
Xhat_omni = ase(n_components=dhat).fit_transform(omni_mtx)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 9.5), gridspec_kw={"width_ratios": [2, 1]})

hm = lined_heatmap(omni_mtx, binary=False, cmap=omni_cmap, cbar=False,
                   title="(A) Omnibus matrix", center=None, alpha=0, ax=axs[0])
sns.despine(ax=hm, top=False, bottom=False, left=False, right=False)
for i in np.arange(8)*1/8:
    hm.vlines(len(omni_mtx)*i, 0, len(omni_mtx), colors="black", lw=.9, linestyle="dashed", alpha=1)
    hm.hlines(len(omni_mtx)*i, 0, len(omni_mtx), colors="black", lw=.9, linestyle="dashed", alpha=1)
for i in np.arange(1, 16, 2)*1/16:
    hm.vlines(len(omni_mtx)*i, 0, len(omni_mtx), colors="black", lw=.9, linestyle="dashed", alpha=.2)
    hm.hlines(len(omni_mtx)*i, 0, len(omni_mtx), colors="black", lw=.9, linestyle="dashed", alpha=.2)
hm.vlines(len(omni_mtx)*1/2, 0, len(omni_mtx), colors="black", lw=1.3, linestyle="solid")
hm.hlines(len(omni_mtx)*1/2, 0, len(omni_mtx), colors="black", lw=1.3, linestyle="solid")

lpm_heatmap(Xhat_omni, ax=axs[1], title="(B) Estimated latent positions", xtitle="Latent dimension")
fig.tight_layout()
fig.savefig("Figures/omni_mtx.{}".format(mode))

In [ ]:
M = len(networks)
n = len(networks[0])

# obtain a M x n x d tensor
Xhat_tensor = Xhat_omni.reshape(M, n, -1)
# the estimated latent positions for the first network
Xhat_human1 = Xhat_tensor[0,:,:]

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(18, 8))

for i in range(4):
    plot_latents(Xhat_tensor[i, :, :], ax=axs[0, i], labels=labels, palette={"L": "#000000", "R": "#999999"},
                 title="Human {:d}".format(i + 1), s=20)
    axs[0,i].get_legend().remove()
    axs[0,i].set_xlim((Xhat_tensor[:,:,0].min(), Xhat_tensor[:,:,0].max()))
    axs[0,i].set_ylim((Xhat_tensor[:,:,1].min(), Xhat_tensor[:,:,1].max()))
    axs[0,i].set_xlabel("Dimension 1")
    axs[0,i].set_ylabel("Dimension 2")
    
for i in range(4):
    plot_latents(Xhat_tensor[i+4, :, :], ax=axs[1, i], labels=labels, palette={"L": "#000000", "R": "#999999"},
                 title="Alien {:d}".format(i + 1), s=20)
    if i != 3:
        axs[1,i].get_legend().remove()
    axs[1,i].set_xlim((Xhat_tensor[:,:,0].min(), Xhat_tensor[:,:,0].max()))
    axs[1,i].set_ylim((Xhat_tensor[:,:,1].min(), Xhat_tensor[:,:,1].max()))
    axs[1,i].set_xlabel("Dimension 1")
    axs[1,i].set_ylabel("Dimension 2")
    
fig.tight_layout()
fig.savefig("Figures/omni_ind.{}".format(mode))

In [ ]:
Phat_hum1 = Xhat_human1 @ Xhat_human1.T